In [1]:
import pandas as pd
import re
import os
import numpy as np
import datetime
from datetime import datetime as dt

In [2]:
csv_list = []

LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/pm4pyexample/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/tch/'
#LUIGI_LOG_PATH='/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/october/mongo/'
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
appended_df = pd.DataFrame()
#appended_df
csv_list.sort()
csv_list

['coma-running-example-just-two-cases.csv',
 'running-example-just-two-cases.csv',
 'semicolon-running-example-just-two-cases.csv']

In [3]:
from difflib import SequenceMatcher 
  
def longestSubstring(str1,str2): 
  
     # initialize SequenceMatcher object with  
     # input string 
     seqMatch = SequenceMatcher(None,str1,str2) 
  
     # find match of longest sub-string 
     # output will be like Match(a=0, b=0, size=5) 
     match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
  
     # print longest substring 
     if (match.size!=0): 
          result = str1[match.a: match.a + match.size]
          print (result)  
          return result
     else: 
          print ('No longest common sub-string found')

In [4]:
LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/pm4pyexample/'

print(os.listdir(LUIGI_LOG_PATH))

filename = 'semicolon-running-example-just-two-cases.csv'
if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        print('Preprocessing... ', filename)
        df = pd.read_csv(log_path, index_col=None, sep=';')
        df['timestamp'] = df[['dd-MM-yyyy:HH.mm']]
        df['activity'] = df[['Activity']]
        df['case'] = df[['Case ID']]
        df = df[['timestamp','activity','state', 'case']]

        
        appended_df = appended_df.append(df)
        print(len(appended_df))
appended_df

['running-example-just-two-cases.xes', 'coma-running-example-just-two-cases.csv', 'just_two_cases_xes_im.png', 'just_two_cases_im.png', 'running-example-just-two-cases.csv', 'semicolon-running-example-just-two-cases.csv', 'just_two_cases_csv_im.png']
Preprocessing...  semicolon-running-example-just-two-cases.csv
20


,timestamp,activity,state,case
0,30-12-2010:11.02,register request,start,1
1,30-12-2010:11.20,register request,end,1
2,30-12-2010:10.06,examine thoroughly,start,1
3,31-12-2010:10.06,examine thoroughly,end,1
4,05-01-2011:15.12,check ticket,start,1
5,06-01-2011:15.12,check ticket,end,1
6,06-01-2011:11.18,decide,start,1
7,07-01-2011:14.24,decide,end,1
8,07-01-2011:14.14,reject request,start,1
9,07-01-2011:14.24,reject request,end,1


In [5]:
df = appended_df
df = df[['activity','state','timestamp','case']]

df['start_time'] = np.where(df['state']=='start', df['timestamp'], np.nan)
df['end_time'] = np.where(df['state']=='end', df['timestamp'], np.nan)

start_df = df[df['state']=='start']
print('Start entries: ',len(start_df))

end_df = df[df['state']=='end']
print('End entries: ',len(end_df))

start_df.head()

Start entries:  10
End entries:  10


/usr/local/trustyou/home/andream/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/trustyou/home/andream/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,activity,state,timestamp,case,start_time,end_time
0,register request,start,30-12-2010:11.02,1,30-12-2010:11.02,NaN
2,examine thoroughly,start,30-12-2010:10.06,1,30-12-2010:10.06,NaN
4,check ticket,start,05-01-2011:15.12,1,05-01-2011:15.12,NaN
6,decide,start,06-01-2011:11.18,1,06-01-2011:11.18,NaN
8,reject request,start,07-01-2011:14.14,1,07-01-2011:14.14,NaN


In [6]:
def convertTimeFormat(date_str): 
    # To match format for pointTransformer.ipynb
    date_obj = dt.strptime(date_str, '%d-%m-%Y:%H.%M')
    result = dt.strftime(date_obj, '%Y-%m-%d %H:%M:%S')
    return result

In [12]:
tt = pd.merge(start_df, end_df, on=['activity', 'case'], how='outer')
tt = tt[tt['start_time_x'].notnull()]
tt = tt[tt['end_time_y'].notnull()]

tt['start_time'] = tt['start_time_x']
tt['end_time'] = tt['end_time_y']

tt['start_time'] = tt.apply(lambda row: convertTimeFormat(row['start_time']), axis=1)
tt['end_time'] = tt.apply(lambda row: convertTimeFormat(row['end_time']), axis=1)
tt['case'] = tt.apply(lambda row: str(row['case']), axis=1)


catt = tt[['case', 'activity','start_time','end_time']]
print(type(catt['case'][0]))
print(len(catt))
catt.head()

<class 'str'>
10


,case,activity,start_time,end_time
0,1,register request,2010-12-30 11:02:00,2010-12-30 11:20:00
1,1,examine thoroughly,2010-12-30 10:06:00,2010-12-31 10:06:00
2,1,check ticket,2011-01-05 15:12:00,2011-01-06 15:12:00
3,1,decide,2011-01-06 11:18:00,2011-01-07 14:24:00
4,1,reject request,2011-01-07 14:14:00,2011-01-07 14:24:00


In [9]:
OUTPUT_CATT_PATH = LUIGI_LOG_PATH+'/../catts_'+filename
print(len(catt))
print(OUTPUT_CATT_PATH)
catt.to_csv(OUTPUT_CATT_PATH, index = None, header=True)

10
/usr/local/trustyou/home/andream/nfs/processmining/pm4pyexample//../catts_semicolon-running-example-just-two-cases.csv


'2010-12-30 11:02:00'